In [1]:
import gdal
from matplotlib import pyplot as plt
import numpy as np
import os
import re
import pandas as pd

# Declare the MCD15A2H folder

In [2]:
"""
Collection dir is assumed to contain a folder of each year of the product

"""
OUTPUTDIR='tilesDIR'
DATADIR='/home/gbessardon/DATA'
SHORTNAME='MCD15A2H'
COLLECTION='061'
collectiondir=os.path.join(DATADIR,SHORTNAME,COLLECTION)
years=['2020', '2021', '2018', '2019','2017']

# Function to read the hdf file metadata

In [3]:
def extract_metadata(ds):
    geoT = ds.GetGeoTransform()
    projT = ds.GetProjection()
    w=ds.RasterXSize
    h=ds.RasterYSize
    offset=0
    scale_factor=1
    fillvalue=9999
    for item,value in ds.GetMetadata_Dict().items():
        if item=='add_offset':
            offset=int(value)
        if item=='scale_factor':
            scale_factor=float(value)
        if item=='_FillValue':
            fillvalue=int(value)
    return(offset,scale_factor,fillvalue,geoT,projT,w,h)

# Function to clean the data

## Function to get the corresponding integer to the QC bit-string

In [4]:
def Getvalidqcinteger(bit57=['000','001'],bit34=['00','11'],
                      bit2=['0'],bit1=['0','1'],bit0=['0']):
    valbit=[]
    valint=[]
    for b57 in bit57:
        for b34 in bit34:
            for b2 in bit2:
                for b1 in bit1:
                    for b0 in bit0:
                        bit=b57+b34+b2+b1+b0
                        qc=int(bit,2)
                        valbit.append(bit)
                        valint.append(qc)
    return(valbit,valint)

## Function to apply the filter

In [5]:
def clean_data(Ar,QCarray,maxvalue,validQCvalues,add_offset=0,scale_factor=1,fillvalue=9999,ECOSGscale_factor=100):
    Arm=Ar*0
    Armasked=Ar
    Armasked[Ar>maxvalue]=fillvalue
    for val in validQCvalues:
        Arm[QCarray==val]=1
    
    Armasked[Arm==0]=fillvalue
    Armasked = scale_factor * (Armasked - add_offset)*ECOSGscale_factor
    new_fill=scale_factor*(fillvalue- add_offset)*ECOSGscale_factor
    return(Armasked.astype(int),new_fill)

## Gather all data treatment

In [6]:
def onefilefilter_treatment(fn,laiband=1,QCband=2,maxlai=100,fillvalue=255,ECOSGsf=100):
    sds=gdal.Open(fn).GetSubDatasets()
    dslai=gdal.Open(sds[laiband][0])
    dsQC=gdal.Open(sds[QCband][0])
    bites,QCv=Getvalidqcinteger()
    of,sf,fil,geoT,projT,w,h=extract_metadata(dslai)
    LAI=dslai.ReadAsArray()
    QCar=dsQC.ReadAsArray()
    LAImasked,filln=clean_data(LAI,QCar,maxlai,QCv,
                               add_offset=of,scale_factor=sf,fillvalue=fil,ECOSGscale_factor=ECOSGsf)
    return (LAImasked,filln,geoT,projT,w,h,of,sf)

# Functions to Create raster from numpy array

In [7]:
def create_raster(output_path,columns,rows,nband = 1,gdal_data_type = gdal.GDT_Int16, driver = r'GTiff'):
    ''' returns gdal data source raster object

    '''
    # create driver
    driver = gdal.GetDriverByName(driver)

    output_raster = driver.Create(output_path,
                                  int(columns),
                                  int(rows),
                                  nband,
                                  eType = gdal_data_type)    
    return output_raster

In [8]:
def numpy_array_to_raster(output_path,
                          numpy_array,
                          geoTin,
                          projectionin,
                          nband = 1,
                          no_data = -9999,
                          gdal_data_type = gdal.GDT_Int16,
                          driver = r'GTiff'):
    ''' returns a gdal raster data source

    keyword arguments:

    output_path -- full path to the raster to be written to disk
    numpy_array -- numpy array containing data to write to raster
    geoTin -- geotransform input from ds.GetGetGeoTransform()
    projectionin -- projection input from ds.GetProjection()
    nband -- the band to write to in the output raster
    no_data -- value in numpy array that should be treated as no data
    gdal_data_type -- gdal data type of raster (see gdal documentation for list of values)
    driver -- string value of the gdal driver to use

    '''


    rows, columns = numpy_array.shape

    # create output raster
    output_raster = create_raster(output_path,
                                  int(columns),
                                  int(rows),
                                  nband,
                                  gdal_data_type) 

    geotransform = geoTin


    output_raster.SetProjection(projectionin)
    output_raster.SetGeoTransform(geotransform)
    output_band = output_raster.GetRasterBand(1)
    output_band.SetNoDataValue(no_data)
    output_band.WriteArray(numpy_array)          
    output_band.FlushCache()
    output_band.ComputeStatistics(False)

    if os.path.exists(output_path) == False:
        raise Exception('Failed to create raster: %s' % output_path)

    return  

# Get the list of all the files

In [9]:
def Create_filelist(collectiondir,years):
    filelistMCD15A2H=[]
    for y in years:
        if os.path.exists(os.path.join(collectiondir,y)) == True:
            for f in os.listdir(os.path.join(collectiondir,y)):
                filelistMCD15A2H.append(os.path.join(collectiondir,y,f))
    return (filelistMCD15A2H)

## Get the list of tiles in the existing files

In [10]:
def Create_tilelist(filelist,tilepattern='.h\d\dv\d\d'):
    tilelist= [re.search(tilepattern,f).group() for f in filelist]
    hlist=[int(t.split('h')[1].split('v')[0]) for t in tilelist]
    vlist=[int(t.split('h')[1].split('v')[1]) for t in tilelist]
    return(tilelist,vlist,hlist)

## Get the day list in the MODIS files by identifying the pattern AYYYYJJJ then gathering then in the ECOSG 10-days classes

In [11]:
def Create_10dayslist(filelist,dayspattern='.A\d\d\d\d\d\d\d',
                     ECOSGdates=['0105', '0115', '0125', '0205', '0215', '0225', '0305', '0315',
                                 '0325', '0405', '0415', '0425', '0505', '0515', '0525', '0605',
                                 '0615', '0625', '0705', '0715', '0725', '0805', '0815', '0825',
                                 '0905', '0915', '0925', '1005', '1015', '1025', '1105', '1115',
                                 '1125', '1205', '1215', '1225']):
    dayslist= [re.search(dayspattern,f).group() for f in filelist]
    yearlist=[int(d[2:6]) for d in dayslist]
    juliandays=[int(d[6:9]) for d in dayslist]
    julian10daysclass=[int((j/365)*36) for j in juliandays]
    ECOSG10days=[ECOSGdates[j10] for j10 in julian10daysclass]
    return(yearlist,juliandays,julian10daysclass,ECOSG10days)

## Gather the previous function and extract the data in a dataframe

In [12]:
def Create_dataframe(collectiondir,years):
    filelist=Create_filelist(collectiondir,years)
    tilelist,vlist,hlist=Create_tilelist(filelist)
    yearlist,juliandays,julian10daysclass,ECOSG10days=Create_10dayslist(filelist)
    data = {'filename':filelist, 'tile':tilelist,'htile':hlist,'vtile':vlist,
            'year':yearlist,'julianday':juliandays,'julian10days':julian10daysclass,
            'ecosg10':ECOSG10days}
    df=pd.DataFrame(data)
    groupeddftilejuliandays=df.groupby(['htile','vtile','ecosg10'])
    return(df,groupeddftilejuliandays)

## Function to create the output dir if it does not exists

In [13]:
def Createoutputdir(outputdir):
    outpath=os.path.join(os.getcwd(),outputdir)
    if not os.path.isdir(outpath):
        os.mkdir(outpath)
    return outpath

# Main part

## Create dataframe with the different files available and sort them by tile and 10days period

In [14]:
df,groupeddftilejuliandays=Create_dataframe(collectiondir,years)

## Create output directory if does not exist

In [15]:
outpath=Createoutputdir(OUTPUTDIR)


## Loop over the group aplly quality filter and calculate the 10-day mean

In [17]:
files_issuelist=[]
for name, group in groupeddftilejuliandays:
        LAIlist=[]
        print(name)
        for i,fn in enumerate(group.filename):
            try:
                (LAImasked,filln,geoT,projT,w,h,of,sf)=onefilefilter_treatment(fn)
                LAIlist.append(np.ma.masked_array(LAImasked,LAImasked==filln))
            except:
                files_issuelist.append(fn)
        LAI_median=np.ma.median(np.ma.stack(LAIlist),axis=0,overwrite_input=True)
        numpy_array_to_raster(os.path.join(outpath,'LAI'+str(name[0])+'v'+str(name[1])+'jd'+str(name[2])+'.tif'),
                              LAI_median,geoT,projT,nband = 1,no_data = filln,gdal_data_type = gdal.GDT_Int16,driver = r'GTiff')

(11, 2, '0715')
(11, 2, '0805')
(11, 2, '0825')
(12, 1, '0715')
(12, 1, '0725')
(12, 1, '0805')
(12, 1, '0815')
(12, 1, '0825')
(12, 2, '0625')
(12, 2, '0715')
(12, 2, '0725')
(12, 2, '0805')
(12, 2, '0825')
(13, 1, '0715')
(13, 1, '0725')
(13, 1, '0805')
(13, 1, '0815')
(13, 1, '0825')
(13, 2, '0625')
(13, 2, '0725')
(13, 2, '0815')
(13, 2, '0825')
(14, 1, '0715')
(14, 1, '0725')
(14, 1, '0805')
(14, 1, '0815')
(14, 1, '0825')
(14, 2, '0625')
(14, 2, '0715')
(14, 2, '0725')
(14, 2, '0805')
(14, 2, '0815')
(15, 1, '0715')
(15, 1, '0725')
(15, 1, '0805')
(15, 1, '0815')
(15, 1, '0825')
(15, 2, '0715')
(15, 2, '0725')
(15, 2, '0805')
(15, 2, '0815')
(15, 2, '0825')
(16, 0, '0715')
(16, 0, '0725')
(16, 0, '0805')
(16, 0, '0815')
(16, 0, '0825')
(16, 1, '0715')
(16, 1, '0725')
(16, 1, '0805')
(16, 1, '0815')
(16, 1, '0825')
(16, 2, '0625')
(16, 2, '0715')
(16, 2, '0725')
(16, 2, '0805')
(16, 2, '0815')
(16, 2, '0825')
(17, 0, '0715')
(17, 0, '0725')
(17, 0, '0805')
(17, 0, '0815')
(17, 0, 